# 1. Get genes from reference genomes of Mflo, W1, Mjav, Mare, Mkon and Ment
Using the genome assemblies and their gff files, make a file of genes (introns included), to be used as mitobim baits for the Mflo read, the reads from the W1 mitochondrial clade, the Mjav mitochondtial clade and Mare respectively.



## 1.1 Annotation files
Only gff3 can go into this dictionary. On occassions where gff2 is available from exonerate, it is dealt with locally with an addapted script.

In [ ]:
# These paths will not work on BioPC8
# which has a unique file structure

root = './annotation/'


mflo_gff = None
w1_gff = root + 'mincognita.augustus.gff3'
vw4_gff = root + 'mjavanicaVW4.augustus.gff3'
Mare_gff = root + 'marenaria.augustus.gff3'
Mkon_gff = None
Ment_gff = root + 'menterolobii.augustus.gff3'
MfloSJF1_gff = root + 'mfloridensis.augustus.gff3'

annotation_files = {'MfloJB5_ref':mflo_gff,
                    'MfloSJF1_ref': MfloSJF1_gff,
                    'MjavVW4_ref':vw4_gff,
                    'MincW1_ref':w1_gff,
                    'MareHarA_ref':Mare_gff,
                    #'Mkon_ref':Mkon_gff,
                    'MentL30_ref':Ment_gff}

## 1.2 Genome assembly files

In [18]:
#!gzip -d ./meloidogyne_assemblies/*.fa*gz

root = './meloidogyne_assemblies/'

mflo_ass = './meloidogyne_assemblies/meloidogyne.velvet.fa'
w1_ass = './meloidogyne_assemblies/mincognita_gapClosed.fa'
vw4_ass = './meloidogyne_assemblies/mjavanica_gapClosed.fa'
Mare_ass = './meloidogyne_assemblies/HarA_gapClosed.fa'
#Mkon_ass = './meloidogyne_assemblies/Kon_gapClosed.fa'
Ment_ass = "/media/amir/DATA/Read_data/assemblies/Ent_gapClosed.fa"
MfloSJF1_ass = root+"MfloSJ1.scf.fasta"
    
assembly_files = {'MfloJB5_ref':mflo_ass,
                  'MfloSJF1_ref': MfloSJF1_ass,
                  'MjavVW4_ref':vw4_ass,
                  'MincW1_ref':w1_ass,
                  'MareHarA_ref':Mare_ass,
                  #'Mkon_ref': Mkon_ass,
                  'MentL30_ref': Ment_ass}

## 1.3 Write gene, CDS and Prot fasta files
### 1.3.1 collect the sequences from the assembly and write to fasta

In [ ]:
from misc import *
import os, gc, sys

from Bio import SeqIO
# make output directories
for fpath in ['protein_ref_file',
              'cds_ref_files',
              'gene_ref_files']:
    makedir(fpath)
    
for ref in annotation_files:
    annotation = annotation_files[ref]
    assembly = assembly_files[ref]
    
    # annotation and assembly exist?
    if annotation and assembly:
        
        prot_out = 'protein_ref_file/'+ref+'.aa.fasta'
        cds_out = 'cds_ref_files/'+ref+'.cds.fasta'
        gene_out = 'gene_ref_files/'+ref+'.nt.fasta'
        
        # output files exists?
        prot_exists = os.path.exists(prot_out)
        cds_exists = os.path.exists(cds_out)
        gene_exists = os.path.exists(gene_out)
        
        # Do we need to make more output files?
        if all([prot_exists,cds_exists,gene_exists]):
            continue
        
        
        gene_hndl = None
        cds_hndl = None
        prot_hndl = None
        
        if not gene_exists:
            gene_hndl = open(gene_out,'wt')
        
        if not cds_exists:
            cds_hndl = open(cds_out,'wt')
        
        if not prot_exists:
            prot_hndl = open(prot_out,'wt')
            
        records = SeqIO.to_dict(SeqIO.parse(assembly,'fasta'))
        
        annotation_hndl = open(annotation,'r')
        
        gene_id = None
        cds_seq = None
        gene_seq = None
        
        for gff in annotation_hndl:
            contig, a, featuretype, start, end, b, strand, c, qualifiers = gff.split('\t')
                
            start = int(start)
            end = int(end)

            qualifiers = {q.split('=')[0]: q.split('=')[1] for
                              q in qualifiers.rstrip().split(';') if q}
                
            start -= 1
            
            if not 'ID' in qualifiers:
                continue
                
            if not gene_id and featuretype == 'gene':
                gene_id = qualifiers["ID"]
                gene_seq = records[contig].seq[start:end]
                if strand == '-':
                    gene_seq = gene_seq.reverse_complement()
                gene_hndl.write('>%s\n%s\n'%(gene_id, str(gene_seq)))
            
            elif (gene_id and 
                not gene_id == qualifiers['ID'].split('.')[0] and 
                featuretype == 'gene'):
                
                cds_hndl.write('>%s\n%s\n'%(gene_id, str(cds_seq)))
                prot_seq = str(cds_seq.translate())
                if prot_seq.endswith('*'):
                    prot_seq = prot_seq[:-1]
                prot_hndl.write('>%s\n%s\n'%(gene_id, prot_seq))
                
                cds_seq = None
                prot_seq = None
                
                gene_id = qualifiers["ID"]
                gene_seq = records[contig].seq[start:end]
                if strand == '-':
                    gene_seq = gene_seq.reverse_complement()
                gene_hndl.write('>%s\n%s\n'%(gene_id, str(gene_seq)))
                
            elif featuretype == 'CDS' and qualifiers['ID'].split('.')[0] == gene_id:
                if not cds_seq:
                    cds_seq = ''
                exon_seq = records[contig].seq[start:end]
                if strand == '+':
                    cds_seq += exon_seq
                elif strand == '-':
                    cds_seq = exon_seq.reverse_complement() + cds_seq
                else:
                    raise RuntimeError('dont know strand %s'%strand)
        if cds_seq:
            cds_hndl.write('>%s\n%s\n'%(gene_id, str(cds_seq)))
            prot_seq = str(cds_seq.translate())
            if prot_seq.endswith('*'):
                prot_seq = prot_seq[:-1]
            prot_hndl.write('>%s\n%s\n'%(gene_id, prot_seq))
            
                
   
        
        # close the output files
        
        annotation_hndl.close()
                                    
        if not gene_hndl:
            gene_hndl.close()
        
        if not cds_hndl:
            cds_hndl.close()
        
        if not prot_hndl:
            prot_hndl.close()
                    
        # clear the memory
        del records
        gc.collect()

## 1.4 Take a different approach with Mflo which is gff2
The gff2 for Mflo was made by running exonerate with cdss as queries. It was done for the mapping analysis. Since the CDS and protein files already exist from Lunt et al 2014, and these CDSs were used to make the gff2 to beginwith, there is no need to extract proteins and cdss again, only genes, from the gff2.
### 1.4.1 Copy the Mflo CDS, protein and gff files from Lunt et al. 2014:

In [ ]:
import os
# The copied files are already in the destination inside this repository

if not os.path.exists('cds_ref_files/MfloJB5_ref.cds.fasta'):
    !cp ../GS978784/mf.cds.fna cds_ref_files/MfloJB5_ref.cds.fasta
if not os.path.exists('protein_ref_file/MfloJB5_ref.aa.fasta'):
    !cp ../GS978784/mf.protein.faa protein_ref_file/MfloJB5_ref.aa.fasta
if not os.path.exists('annotations/MfloJB5.gff'):
    !cp ../GS978784/mf.cds.fna annotation/MfloJB5.gff

### 1.4.2 Extract full genes from the MfloJB5 assembly using the gff2

In [ ]:
import os
from Bio import SeqIO

mflo_gff2 = 'annotation/MfloJB5.gff'

# mflo assembly:
records = SeqIO.to_dict(SeqIO.parse(assembly_files['MfloJB5_ref'],'fasta'))

mflo_genes_fpath = 'gene_ref_files/MfloJB5_ref.nt.fasta'

if not os.path.exists(mflo_genes_fpath):

    hndl = open(mflo_genes_fpath,'wt')

    with open(mflo_gff2,'r') as lines:
        for line in lines:
            contig, a, featuretype, start, end, b, strand, c, qualifiers = line.split('\t')
            if featuretype == 'gene':
                start = int(start)
                end = int(end)
                
                # gff2 style
                qualifiers = {q.split()[0]: q.split()[1] for
                              q in qualifiers.rstrip().split(' ; ') if q}
                start -= 1
                
                gene_id = qualifiers['sequence']
                gene_seq = records[contig].seq[start:end]
                if strand == '-':
                    gene_seq = gene_seq.reverse_complement()
                hndl.write('>%s\n%s\n'%(gene_id, str(gene_seq)))


    hndl.close()

## 1.5 Annotate unannotated genomes with exonerate 
All the denovo assembled genomes (ie reference genomes) now have an augustus gff3 annotation and section 1.5 is obsolete. The functions used here may still be usfull at some point so I keep it in the notebook.

### 1.5.1 Prerequisites to run and  parse exonerate output
These will run a multithread exonerate analysis using the protein2genome model and will parse the spesific output into a cds file, a gene file and a gff2 file

In [ ]:
from misc import *
from StringIO import StringIO


def run_exonerate(q, t, outdir, threads):
    
    import time, threading
    from glob import glob
    from warnings import warn
    from subprocess import Popen

    makedir(outdir)

    # The exonerate command line

    # chunk the query file
    a = "exonerate --querychunkid %i --querychunktotal %i "
    # format the output into gene and CDS fasta (exonerate roll your own syntax
    # plus proper Python escapes: backslash for quotation and backslash,
    # percent for percent)
    b = "--ryo \"STARTGENE\\n>%%qi\\n%%tas\\nENDGENE\\nSTARTCDS\\n>%%qi\\n%%tcs\\nENDCDS \" "
    # gff
    c = "--showtargetgff "
    # model and limit to number of matches
    d = "-m protein2genome "#-n 1 "
    # query and target file paths
    e = "-q %s -t %s "%(q,t)
    # output fpath
    f = "> %s"%outdir+"/%s"

    exonerate_cline = a+b+c+d+e+f

    class MyThread(threading.Thread):
        def __init__(self, querychunkid, querychunktotal, out):
            threading.Thread.__init__(self)
            self.querychunkid = querychunkid
            self.querychunktotal = querychunktotal
            self.out = out

        def run(self):
            cline = exonerate_cline%(self.querychunkid,
                                     self.querychunktotal,
                                     self.out)
            p = Popen(cline,shell=True)
            p.communicate()


    # outputs exist?
    outputs_exist = len(list(glob('%s/exoout*'%outdir))) > 0

    if outputs_exist:
        warn('some output exists. Skipping')
    else:
        threads = [MyThread(i, threads, 'exoout%i'%i) for i in range(1, threads+1)]

        for t in threads:
            t.start()

        for t in threads:
            t.join()
            
        print "exonerate done"


def parse_a_single_record(record):
    
    output = {}
    
    def parse_line_element(ind, parter, element_name):
        elementline = record[ind]
        try:
            output[element_name] = elementline.rstrip().split(parter)[1]
            if ' [revcomp]' in elementline:
                output[element_name] = output[element_name].replace(' [revcomp]','')
            elif ' -> ' in output[element_name] and \
                 element_name == 'Target_start':
                output[element_name] = output[element_name].split(' -> ')[0]
            elif ' -> ' in output[element_name] and \
                 element_name == 'Target_end':
                output[element_name] = output[element_name].split(' -> ')[1]
            try:
                output[element_name] = int(output[element_name])
            except:
                pass
        except:
            raise RuntimeError("%s not found"%parter)

            
    line_elements = [[2, 'Query: ', 'Query'],
                     [3, 'Target: ', 'Target'],
                     [5, 'Raw score: ', 'Score'],
                     [7, 'Target range: ', 'Target_start'],
                     [7, 'Target range: ', 'Target_end']]
    
    
    for ind, parter, element_name in line_elements:
        parse_line_element(ind, parter, element_name)
        
    def parse_block_element(init, halt, element_name):
        element = ""
        get = False
        for line in record:
            if init in line:
                get = True
            elif halt in line:
                get = False
            elif get:
                element += line
        output[element_name] = element
        
    block_elements = [['STARTGENE','ENDGENE','Gene'],
                      ['STARTCDS','ENDCDS','CDS'],
                      ['--- START OF GFF DUMP ---',
                       '--- END OF GFF DUMP ---',
                       'GFF']
                     ]
    for init, halt, element_name in block_elements:
        parse_block_element(init, halt, element_name)
        
    return output

def records_overlapping(record1,record2):
    
    from misc import is_overlapping
    
    contig1 = record1['Target']
    contig2 = record2['Target']
    if not contig1 == contig2:
        return False, None
    
    coords1 = (record1['Target_start'], record1['Target_end'])
    coords2 = (record2['Target_start'], record2['Target_end'])
    
    if not is_overlapping(coords1, coords2):
        return False, None
    
    if record1['Score'] >= record2['Score']:
        return True, 0
    else:
        return True, 1
    
def iter_records(fpath):
    with open(fpath,'r') as lines:
        record = []
        get = False
        for line in lines:
            if 'C4 Alignment:' in line:
                get = True
            if get and 'C4 Alignment:' in line:
                if len(record) > 0:
                    yield record
                    record = [line]
                else:
                    record = [line]
            elif get:
                record.append(line)
        yield record
        
def filter_out_short_exonerate_matches(exonerate_records_with_CDS, query_lengths, len_cutoff=0.95, n_cutoff = 0.1):
    """
    exonerate_records_with_CDS is a list of exonerate records 
    each parsed with parse_a_single_record.
    Individual records can be passed to parse_a_single_record
    from an exonerate output file with iter_records
    The output file has to come from an exonerate run in which
    the following ryo was used:
    --ryo \"STARTGENE\\n>%%qi\\n%%tas\\nENDGENE\\nSTARTCDS\\n>%%qi\\n%%tcs\\nENDCDS \"
    (the first \ or % are python excapes and should be removed in the command line)
    """
    records_to_remove = set()
    for i in range(len(exonerate_records_with_CDS)):
        cdsfastahndl = StringIO(exonerate_records_with_CDS[i]['CDS'])
        seqrecord = SeqIO.read(cdsfastahndl,'fasta')
        length = len(seqrecord.seq)
        if length < query_lengths[exonerate_records_with_CDS[i]['Query']]*len_cutoff:
            records_to_remove.add(i)
        elif str(seqrecord.seq).count('N') > query_lengths[exonerate_records_with_CDS[i]['Query']]*n_cutoff:
            records_to_remove.add(i)
    records_to_remove = sorted(records_to_remove, reverse=True)
    for i in records_to_remove:
        exonerate_records_with_CDS.pop(i)
    return exonerate_records_with_CDS
        
def iter_half_matrix_indices(iterable):
    """
    >>> iterable = [1,2,3]
    >>> for i,j in iter_half_matrix_indices(iterable):
    >>>     print i, j
    0 1
    0 2
    1 2
    """
    length = len(iterable)
    for i in range(len(iterable)):
        for j in range(i+1, len(iterable)):
            yield (i, j)
            
def remove_overlapping_records(exonerate_records_from_single_contig, contig):
    
    records_to_remove = set()

    for i,j in iter_half_matrix_indices(exonerate_records_from_single_contig):
        ovrlap, which =\
        records_overlapping(exonerate_records_from_single_contig[j],
                            exonerate_records_from_single_contig[i])

        if ovrlap:
            if which == 1:
                records_to_remove.add(j)
            else:
                records_to_remove.add(i)

    if len(records_to_remove) > 0:
        try:
             assert(max(records_to_remove)+1 <= len(exonerate_records_from_single_contig))
        except:
            raise AssertionError(contig)

        records_to_remove = sorted(records_to_remove, reverse=True)

        try:
            assert(records_to_remove[0] < len(exonerate_records_from_single_contig))
        except:
            raise AssertionError(contig)

        try:
            assert(records_to_remove[0] == max(records_to_remove))
        except:
            raise AssertionError(contig)

        for i in records_to_remove:
            exonerate_records_from_single_contig.pop(i)
    return exonerate_records_from_single_contig
                
def parse_several_exonerate_output_files(outfiles_dir,
                                         cds_fpath,
                                         gene_fpath,
                                         gff_fpath,
                                         q):
    from glob import glob
    import sys
    from Bio import SeqIO
    from StringIO import StringIO

    outfiles = glob("%s/exoout*"%outfiles_dir)
    
    # q is proteins, we write CDSs
    lengths = {s.id: len(s.seq)*3 for s in SeqIO.parse(q,'fasta')}

    records = []

    for f in outfiles:
        for record in iter_records(f):
            #print record
            r = parse_a_single_record(record)
            records.append(r)


    records = sorted(records,

                     key=lambda r: (r['Target'],
                                    r['Target_end'] - r['Target_start']),

                     reverse = True)


    cds = open(cds_fpath,'wt')   
    gene = open(gene_fpath,'wt')
    gff = open(gff_fpath,'wt')


    contig = records[0]['Target']
    seen_contigs = []

    contig_records = []
    x = 0

    for r in records:
        if r['Target'] == contig:
            contig_records.append(r)

        else:
            
            # Remove matches that are less than 0.95 the query
            
            records_to_remove = set()
            contig_records = filter_out_short_exonerate_matches(contig_records, lengths)
            
            # Remove one of two overlapping matches based on score
            
            contig_records = remove_overlapping_records(contig_records, contig)

            for cr in contig_records:
                cds.write(cr['CDS'].replace('>','>%i.'%x))
                gene.write(cr['Gene'].replace('>','>%i.'%x))
                gff.write("#GlobalCounter%i\n"%x+cr['GFF'])
                x += 1

            seen_contigs.append(contig) 

            if len(seen_contigs) % 1000 == 0:
                print len(seen_contigs)
                sys.stdout.flush()

            contig = r['Target']
            contig_records = [r]

    # sort records for the last contig
    
    # remove short matches
    records_to_remove = set()
    contig_records = filter_out_short_exonerate_matches(contig_records, lengths)

    # Remove one of two overlapping matches based on score

    contig_records = remove_overlapping_records(contig_records, contig)

    for cr in contig_records:
        cds.write(cr['CDS'].replace('>','>%i.'%x))
        gene.write(cr['Gene'].replace('>','>%i.'%x))
        gff.write("#GlobalCounter%i\n"%x+cr['GFF'])
        x += 1

    seen_contigs.append(contig) 

    if len(seen_contigs) % 1000 == 0:
        print len(seen_contigs)
        sys.stdout.flush()

    cds.close()
    gene.close()
    gff.close() 

    assert len(seen_contigs) == len(set(seen_contigs))

### 1.5.2 Run exonerate for SJF1
A preliminary phylogenetic analysis using snps shows that the Nuclear genome of SJF1 is only slightly diverged from that of W1. I am running exonerate to annotate the Mkon assembly with the W1 proteins.

In [ ]:
# Run exonerate
q = 'protein_ref_file/W1_ref.aa.fasta'
t = assembly_files['MfloSJF1_ref']
outdir = "./SJF1_exonerate_outputs/"
threads = 16

# This will be skipped if the output already exists
run_exonerate(q, t, outdir, threads)

### 1.5.3 parse SJF1 exonerate output

In [ ]:
import os
from glob import glob
from warnings import warn

cds_exists = os.path.exists('./cds_ref_files/MfloSJF1_ref.cds.fasta')
gene_exists = os.path.exists('./gene_ref_files/MfloSJF1_ref.nt.fasta')
gff_exists = os.path.exists('./gff_ref_files/MfloSJF1_ref.gff') 
input_exists = len(list(glob('./SJF1_exonerate_outputs/exoout*'))) > 0
q = 'protein_ref_file/W1_ref.aa.fasta'

if any([cds_exists,
        gene_exists, 
        gff_exists, 
        not input_exists]):
    
    warn('Some input is missing or output exists')

else:
    parse_several_exonerate_output_files('./SJF1_exonerate_outputs/',
                                        './cds_ref_files/MfloSJF1_ref.cds.fasta',
                                        './gene_ref_files/MfloSJF1_ref.nt.fasta',
                                        './gff_ref_files/MfloSJF1_ref.gff',
                                        q)

### 1.5.6 Make protein file for MfloSJF1

In [ ]:
import os
from warnings import warn
from Bio import SeqIO

for smpl in ['MfloSJF1']:
    
    cds_file = './cds_ref_files/%s_ref.cds.fasta'%smpl
    prot_file = './protein_ref_file/%s_ref.aa.fasta'%smpl
    
    cds_exists = os.path.exists(cds_file)
    prot_exists = os.path.exists(prot_file)
    
    if not cds_exists:
        warn('input missing')
        continue
        
    if prot_exists:
        warn('output_exists')
        continue
    
    cdss = SeqIO.parse(cds_file,'fasta')
    prot_hndl = open(prot_file,'wt')
    
    for cds in cdss:
    
        prot_seq = str(cds.seq.translate())

        if prot_seq.endswith('*'):
            prot_seq = prot_seq[:-1]

        prot_hndl.write('>%s\n%s\n'%(cds.id, prot_seq))

## 1.6 Quality checks
### 1.6.1 Check the number of protein sequences in the output

In [20]:
from glob import glob
import sys
for f in glob('protein_ref_file/*'):
    print f, len(list(SeqIO.parse(f,'fasta')))
    sys.stdout.flush()
    
for f in glob('cds_ref_files/*'):
    print f, len(list(SeqIO.parse(f,'fasta')))
    sys.stdout.flush()
    
for f in glob('gene_ref_files/*'):
    print f, len(list(SeqIO.parse(f,'fasta')))
    sys.stdout.flush()

protein_ref_file/MincW1_ref.aa.fasta 24714
protein_ref_file/MentL30_ref.aa.fasta 31051
protein_ref_file/MfloJB5_ref.aa.fasta 15114
protein_ref_file/Mkon_ref.aa.fasta 38995
protein_ref_file/MjavVW4_ref.aa.fasta 25697
protein_ref_file/MfloSJF1_ref.aa.fasta 14399
protein_ref_file/MareHarA_ref.aa.fasta 30286
cds_ref_files/MareHarA_ref.cds.fasta 30299
cds_ref_files/MfloSJF1_ref.cds.fasta 14408
cds_ref_files/MfloJB5_ref.cds.fasta 15119
cds_ref_files/MjavVW4_ref.cds.fasta 25697
cds_ref_files/MentL30_ref.cds.fasta 31051
cds_ref_files/Mkon_ref.cds.fasta 38995
cds_ref_files/MincW1_ref.cds.fasta 24714
gene_ref_files/MjavVW4_ref.nt.fasta 25697
gene_ref_files/MfloSJF1_ref.nt.fasta 14414
gene_ref_files/MfloJB5_ref.nt.fasta 14747
gene_ref_files/Mkon_ref.nt.fasta 38995
gene_ref_files/MareHarA_ref.nt.fasta 30304
gene_ref_files/MentL30_ref.nt.fasta 31051
gene_ref_files/MincW1_ref.nt.fasta 24714


### 1.6.2 Remove redundant sequences
Identify clusters of near identical coding sequences and retain only the longest in the cluster, using vsearch

In [22]:
import misc

misc.makedir('cds_ref_centroids/')
vsearch_template = "vsearch --cluster_fast cds_ref_files/{0}.cds.fasta --id 0.98 "
vsearch_template += "--centroids cds_ref_centroids/{0}.cds.fasta --threads 14"

for ref in assembly_files:
    if os.path.exists('cds_ref_centroids/%s.cds.fasta'%ref):
        continue
    cline = vsearch_template.format(ref)
    out, err = misc.execute_cline(cline)

### 1.6.3 confirm CDSs and translations

In [ ]:
from misc import *
from Bio import SeqIO
import os, sys
from warnings import warn

for fpath in ['protein_ref_reviewed',
              'cds_ref_reviewed',
              'gene_ref_reviewed',
              'stopped_protein_ref_reviewed',
              'stopped_cds_ref_reviewed',
              'stopped_gene_ref_reviewed']:
    makedir(fpath)

rejected_genes = []    

for ref in assembly_files:
    
    # input file names
    prot_in = 'protein_ref_file/'+ref+'.aa.fasta'
    cds_in =  'cds_ref_centroids/'+ref+'.cds.fasta'
    gene_in = 'gene_ref_files/'+ref+'.nt.fasta'
    
    # output file names
    prot_out = 'protein_ref_reviewed/'+ref+'.aa.fasta'
    cds_out =  'cds_ref_reviewed/'+ref+'.cds.fasta'
    gene_out = 'gene_ref_reviewed/'+ref+'.nt.fasta'
    stopped_prot_out = 'stopped_protein_ref_reviewed/'+ref+'.aa.fasta'
    stopped_cds_out =  'stopped_cds_ref_reviewed/'+ref+'.cds.fasta'
    stopped_gene_out = 'stopped_gene_ref_reviewed/'+ref+'.nt.fasta'
    
    # files exist?
    prot_in_exists = os.path.exists(prot_in)
    cds_in_exists = os.path.exists(cds_in)
    gene_in_exists = os.path.exists(gene_in)
    
    prot_out_exists = os.path.exists(prot_out)
    cds_out_exists = os.path.exists(cds_out)
    gene_out_exists = os.path.exists(gene_out)
    
    input_files_exist = [prot_in_exists, cds_in_exists, gene_in_exists]
    output_files_exist = [prot_out_exists, cds_out_exists, gene_out_exists]
    
    # Dont try to work on samples with missing inputs
    if not all(input_files_exist):
        warn('some or all input files are missing for ref %s. Skipping'%ref)
        continue
    
    # Don't work on samples with output files
    if all(output_files_exist):
        warn('skipping ref %s. all outputs exist'%ref)
        continue
        
    
    if any(output_files_exist):
        warn('some output files exist for ref %s. Skipping'%ref)
        continue
        
    # Only work on samples with all the inputs and none of the outputs
    if (all(input_files_exist) and not any(output_files_exist)):
        
        # Get the unfiltered genes, cdss and proteins as SeqRecords
        genes = SeqIO.parse('gene_ref_files/%s.nt.fasta'%ref,'fasta')
        cdss = SeqIO.to_dict(SeqIO.parse('cds_ref_files/%s.cds.fasta'%ref,'fasta'))
        proteins = SeqIO.to_dict(SeqIO.parse('protein_ref_file/%s.aa.fasta'%ref,'fasta'))
        
        # File handles for filtered reocrds
        prot_hndl =   open(prot_out,'wt') 
        cds_hndl =    open(cds_out ,'wt')
        gene_hndl =   open(gene_out,'wt')
        stopped_gene_hndl =   open(stopped_gene_out,'wt') 
        stopped_cds_hndl =   open(stopped_cds_out,'wt') 
        stopped_prot_hndl =   open(stopped_prot_out,'wt') 
 
        for gene in genes:
            
            # filter out gene if there's no CDS
            if not gene.id in cdss:
                rejected_genes.append([ref, gene.id,'no cds'])
                continue
                
            # filter out gene if CDS shorter than 90
            if not len(cdss[gene.id]) >= 90:
                continue
            
            # filter out gene if there's no protein
            if not gene.id in proteins:
                rejected_genes.append([ref, gene.id,'no protein'])
                continue
            
            # filter out gene if the CDS seq not identical
            # to the gene, except for gaps
            with open('temp.fas','wt') as tmp:
                tmp.write(gene.format('fasta')+
                          cdss[gene.id].format('fasta'))
                
            align = mafft_align_contigs('temp.fas')
            os.remove('temp.fas')
            cumulative_entropy = get_aln_cumulative_entropy(align)
            if cumulative_entropy > 0:
                rejected_genes.append([ref, gene.id,'cds does not match gene'])
                continue
            
            # filter out gene if cds translation and the protein
            # sequence from the file are not the same
            translated_cds = str(cdss[gene.id].seq.translate())
            if translated_cds.endswith('*'):
                translated_cds = translated_cds[:-1]
            
            if not translated_cds == str(proteins[gene.id].seq):
                rejected_genes.append([ref, gene.id,'protein does not match cds'])
                continue
                
            # filter out gene if the cds has a stop codon in the middle
            # Because I am using proteins for OG clustering and for codon
            # alignment in the phylogenetic analysis, I cannot have genes with
            # point deleteion/ frameshift. This is a shame because I might be 
            # lossing a lot of data, but then a direct sequence alignment of CDSs
            # is to inaccurate to trust without manual review.
            stopped = False
            if '*' in str(proteins[gene.id].seq):
                stopped = True
            
            
            # if all good, write to files
            if stopped:
                stopped_gene_hndl.write(gene.format('fasta'))
                stopped_cds_hndl.write(cdss[gene.id].format('fasta'))
                stopped_prot_hndl.write(proteins[gene.id].format('fasta'))
            else:
                gene_hndl.write(gene.format('fasta'))
                cds_hndl.write(cdss[gene.id].format('fasta'))
                prot_hndl.write(proteins[gene.id].format('fasta'))
    
        prot_hndl.close()
        cds_hndl.close()
        gene_hndl.close()
    
        stopped_prot_hndl.close()
        stopped_cds_hndl.close()
        stopped_gene_hndl.close()
        
        
picdump_compress(rejected_genes,'rejected_genes.pkl')

### 1.6.4 Check the number of protein sequences in the reviewed output

In [24]:
from Bio import SeqIO
from glob import glob
import sys
import pandas as pd

refs = [ref for ref in assembly_files]

summary = [['data']+refs]

file_types = [
    ['gene_ref_files','nt'],
    ['cds_ref_files','cds'],
    ['protein_ref_file','aa'],
    ['cds_ref_centroids','cds'],
    ['gene_ref_reviewed','nt'],
    ['cds_ref_reviewed','cds'],
    ['protein_ref_reviewed','aa'],
    ['stopped_gene_ref_reviewed','nt'],
    ['stopped_cds_ref_reviewed','cds'],
    ['stopped_protein_ref_reviewed','aa']
]


for ftype in file_types:
    counts = [ftype[0]]
    for ref in refs:
        f = '%s/%s.%s.fasta'%(ftype[0],ref,ftype[1])
        counts.append(len(list(SeqIO.parse(f,'fasta'))))
    summary.append(counts)
    
header = summary.pop(0)
df = pd.DataFrame(summary, columns=header)
df

/home/amir/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


,data,MjavVW4_ref,MincW1_ref,MfloJB5_ref,MentL30_ref,MfloSJF1_ref,MareHarA_ref
0,gene_ref_files,25697,24714,14747,31051,14414,30304
1,cds_ref_files,25697,24714,15119,31051,14414,30299
2,protein_ref_file,25697,24714,15114,31051,14414,30286
3,cds_ref_centroids,23870,22871,15055,29781,13871,28219
4,gene_ref_reviewed,19690,18078,14500,22993,10634,21655
5,cds_ref_reviewed,19690,18078,14500,22993,10634,21655
6,protein_ref_reviewed,19690,18078,14500,22993,10634,21655
7,stopped_gene_ref_reviewed,4221,4333,0,5039,2590,6401
8,stopped_cds_ref_reviewed,4221,4333,0,5039,2590,6401
9,stopped_protein_ref_reviewed,4221,4333,0,5039,2590,6401


In [25]:
import misc

misc.makedir('all_gene_ref_reviewed')

cline = "cat gene_ref_reviewed/{0}.nt.fasta "
cline += "stopped_gene_ref_reviewed/{0}.nt.fasta > "
cline += "all_gene_ref_reviewed/{0}.nt.fasta"

for smpl in assembly_files:
    
    misc.execute_cline(cline.format(smpl))

import misc

misc.makedir('all_protein_ref_reviewed')

cline = "cat protein_ref_reviewed/{0}.aa.fasta "
cline += "stopped_protein_ref_reviewed/{0}.aa.fasta > "
cline += "all_protein_ref_reviewed/{0}.aa.fasta"

for smpl in assembly_files:
    
    misc.execute_cline(cline.format(smpl))
    
import misc

misc.makedir('all_cds_ref_reviewed')

cline = "cat cds_ref_reviewed/{0}.cds.fasta "
cline += "stopped_cds_ref_reviewed/{0}.cds.fasta > "
cline += "all_cds_ref_reviewed/{0}.cds.fasta"

for smpl in assembly_files:
    
    misc.execute_cline(cline.format(smpl))    

misc.py:291: UserWarning: keeping existing fpath all_gene_ref_reviewed
  warnings.warn('keeping existing fpath %s'%name)
misc.py:291: UserWarning: keeping existing fpath all_protein_ref_reviewed
  warnings.warn('keeping existing fpath %s'%name)
misc.py:291: UserWarning: keeping existing fpath all_cds_ref_reviewed
  warnings.warn('keeping existing fpath %s'%name)
